In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
import UltraMotionCapture
UltraMotionCapture.output_msg = False  # mute all output message

#### Pipeline Testing Operators

In [ ]:
import time
from UltraMotionCapture import kps, obj3d, obj4d

def pipeline_operator(
    file_vicon: str = '../UltraMotionCapture/data/6kmh_softbra_8markers_1.csv',
    folder_3dmd: str = '../UltraMotionCapture/data/6kmh_softbra_8markers_1/',
    start: int = 0,
    stride: int = 10,
    frame_num: int = 2,
    sample_num: int = 1500,
    scale_rate: float = 1,
    ):
    """tbf"""
    start_time = time.time()

    # load vicon data
    vicon = kps.MarkerSet()
    vicon.load_from_vicon(
        filedir=file_vicon, 
        scale_rate=scale_rate
    )
    vicon.interp_field()

    # load 3dmd data
    o3_ls = obj3d.load_obj_series(
        folder=folder_3dmd,
        start=start,
        end=frame_num * stride - 1,
        stride=stride,
        sample_num=sample_num,
        scale_rate=scale_rate,
        obj_type=obj3d.Obj3d_Deform,
    )

    # init obj4d data and estimate the displacement field
    fps = 120 / stride
    o4 = obj4d.Obj4d_Deform(
        markerset=vicon,
        fps=fps,
        enable_rigid=False,
        enable_nonrigid=True,
    )
    o4.add_obj(*o3_ls)

    # estimate the computational time
    end_time = time.time()
    regist_time = end_time - start_time

    # return error estimation result combined with the computational time
    diff_dict = o4.regist_verify()
    diff_dict['compute_time'] = regist_time
    return diff_dict

In [ ]:
def condition_contrl_testing(
    condition_name: str,
    condition_ls: list,
    ):
    """tbf"""
    dist_mean_ls = []
    dist_std_ls = []
    compute_time_ls = []

    for condition in condition_ls:
        diff_dict = pipeline_operator(**{condition_name: condition})

        dist_mean_ls.append(diff_dict['dist_mean'])
        dist_std_ls.append(diff_dict['dist_std'])
        compute_time_ls.append(diff_dict['compute_time'])

        print('{} = {} running time: {} s'.format(condition_name, condition, diff_dict['compute_time']))

    return dist_mean_ls, dist_std_ls, compute_time_ls

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['figure.dpi'] = 150

def plot_operator(
    condition_str: str,
    condition_ls: list,
    dist_mean_ls: list, 
    dist_std_ls: list, 
    compute_time_ls: list,
    x_axis_mode: str = 'linear'
    ):
    """tbf"""
    fig, axs = plt.subplots(2, 1)

    axs[0].plot(
        condition_ls, 
        dist_mean_ls
        )

    axs[0].errorbar(
        condition_ls,
        dist_mean_ls,
        yerr = dist_std_ls,
        fmt ='o',
        capsize=5)

    axs[1].plot(
        condition_ls,
        compute_time_ls
        )

    # labels setting
    plt.xlabel(condition_str)
    axs[0].set_ylabel('registration error (mm)')
    axs[0].grid(True)
    axs[1].set_ylabel('computation time (s)')
    axs[1].grid(True)
    
    if x_axis_mode == 'log':
        axs[0].set_xscale('log')
        axs[1].set_xscale('log')

    # fig.show()

#### Factor: Sampling Points

In [ ]:
# testing value
condition_ls = (100, 500, 1000, 1500, 2000, 2500, 3000)
dist_mean_ls, dist_std_ls, compute_time_ls = condition_contrl_testing('sample_num', condition_ls)

In [ ]:
plot_operator('sampling points per point cloud', condition_ls, dist_mean_ls, dist_std_ls, compute_time_ls)

#### Factor: Scaling Rate

In [ ]:
# testing value
condition_ls = (1, 1e-1, 1e-2, 1e-3, 1e-4)
dist_mean_ls, dist_std_ls, compute_time_ls = condition_contrl_testing('scale_rate', condition_ls)

In [ ]:
plot_operator('scaling rate of the point cloud', condition_ls, dist_mean_ls, dist_std_ls, compute_time_ls, x_axis_mode='log')

#### Factor: Frame Stride

In [ ]:
# testing value
condition_ls = (1, 4, 6, 8, 10)
dist_mean_ls, dist_std_ls, compute_time_ls = condition_contrl_testing('stride', condition_ls)

In [ ]:
plot_operator('stride between frames', condition_ls, dist_mean_ls, dist_std_ls, compute_time_ls)